In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

This notebook is used to test the conversion of non-deterministic <span style="font-variant:small-caps;">Fsm</span>s into deterministic <span style="font-variant:small-caps;">Fsm</span>s.

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, RecursiveMap, Tuple } from "recursive-set";
const viz = await instance();

In order to represent the <span style="font-variant:small-caps;">Fsm</span>s graphically, we use the notebook `FSM-2-Dot.ipynb`.  This notebook uses the package `graphviz`.

In [ ]:
import {
    NFA,
    DFA,
    nfa2dfa,
    TransRel,
    State,
    DFAState,
    Char
} from "./01-NFA-2-DFA";
import { nfa2dot, dfa2dot, nfa2string, dfa2string, renderLegend } from "./FSM-2-Dot";

In [ ]:
const stateList = Array.from({ length: 8 }, (_, i) => `q${i}`);
const States = new RecursiveSet<State>(...stateList);
States;

In [ ]:
const Sigma = new RecursiveSet<Char>("a", "b");

In [ ]:
const delta: TransRel = new RecursiveMap();
delta.set(new Tuple("q0", "ε"), new RecursiveSet("q1", "q2"));
delta.set(new Tuple("q5", "ε"), new RecursiveSet("q7"));
delta.set(new Tuple("q6", "ε"), new RecursiveSet("q7"));
delta.set(new Tuple("q7", "ε"), new RecursiveSet("q0"));
delta.set(new Tuple("q1", "b"), new RecursiveSet("q3"));
delta.set(new Tuple("q2", "a"), new RecursiveSet("q4"));
delta.set(new Tuple("q3", "a"), new RecursiveSet("q5"));
delta.set(new Tuple("q4", "b"), new RecursiveSet("q6"));
delta;

The non-deterministic <span style="font-variant:small-caps;">Fsm</span> defined below is taken from the lecture notes.

In [ ]:
const nfa44: NFA = {
    Q: States,
    Σ: Sigma,
    δ: delta,
    q0: "q0",
    A: new RecursiveSet("q7"),
};
nfa2string(nfa44);

The function `nfa2dot`can be used to render this <span style="font-variant:small-caps;">Fsm</span>.

In [ ]:
const dotNFA: string = nfa2dot(nfa44);

In [ ]:
display.html(viz.renderString(dotNFA, { format: "svg" }));

This recognizes the same language as the language described by
$$ (a \cdot b + b \cdot a) \cdot (a \cdot b + b \cdot a)^* $$

Let us convert it into a deterministic Fsm:

In [ ]:
const dfa44: DFA = nfa2dfa(nfa44);
dfa2string(dfa44);

The function `dfa2dot`can be used to render this <span style="font-variant:small-caps;">Fsm</span>.

In [ ]:
const dotDFA = dfa2dot(dfa44);

In [ ]:
display.html(viz.renderString(dotDFA, { format: "svg" }));

In [ ]:
display.html(renderLegend(dfa44))